# Code The Backpropagation


Esta implementação foi baseada no exemplo do notebook backpropagation passo-a-passo, tanto os dados quanto as ideias.

In [118]:
import numpy as np

In [119]:
class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias
        
    def calculate_output(self, inputs):
        self.output = self.activation(self.net(inputs))
        return self.output
        
    # Realiza o calculo de soma(W*x + b)    
    def net(self, inputs):
        return np.dot(self.weights, inputs) + self.bias    
    
    # Função de ativação, no caso estamos utilizando a função logistica f(x) = 1/(1 + exp(-x)) que gera curva sigmoide
    def activation(self, net):
        return 1.0 / (1.0 + np.exp(-net))

    # Derivada da função logistica, se f'(x) = f(x)(1 - f(x))
    def derivative_activation(self):
        return self.output*(1 - self.output)
    
    def calculate_delta(self, target):
        self.delta = (self.output - target)*self.derivative_activation() 
        return self.delta
    
    def update_weights(self, learning_rate, errors_total_weights):
        for i in range(0, len(self.weights)):
            self.weights[i] -= learning_rate * errors_total_weights[i]
        

In [120]:
class NeuralNetwork:
    LEARNING_RATE = 0.5
    
    # Número de entradas de dados, número de nós na camada oculta e número de nós de saida.
    def __init__(self, num_inputs, num_hidden, num_outputs):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        
        # OBS: Dados de teste do artigo que serviu de base
        self.hidden_layer = [ Neuron([.15, .20], .35), Neuron([.25, .30], .35) ]
        self.output_layer = [ Neuron([.40, .45], .60), Neuron([.50, .55], .60) ]
        
        # self.hidden_layer = self.init_layer_weights(num_inputs, num_hidden)
        # self.output_layer = self.init_layer_weights(num_hidden, num_outputs)
        
    # Inicia as camadas de neuronios com pesos gerados aleatoriamente para cada neuronio     
    def init_layer_weights(self, num_weights, num_neurons):
        neurons_layer = []
        for i in range(0, num_neurons):
            weights = np.random.randn(num_weights) * 0.05 
            bias = (np.random.randn(1) * 0.05)[0] 
            neurons_layer.append(Neuron(weights, bias))
        return neurons_layer
        
    def forward_pass(self, inputs):
        outputs_hidden = self.feed_forward(inputs, self.hidden_layer)
        return self.feed_forward(outputs_hidden, self.output_layer)
    
    def backward_pass(self, targets):
        """
            TODO: 
        """

    # calculando deltas para a camada de saida (output layer)    
    def compute_output_deltas(self, targets):
        for neuron, target in zip(self.output_layer, targets):
            neuron.calculate_delta(target)
        
    # Atualização dos pesos, só pode ser executado se o calculo dos deltas for realizado antes    
    def update_output_weights(self):
        for o_neuron in self.output_layer:
            for h_neuron, o_weights in zip(self.hidden_layer, o_neuron.weights):
                errors_total_weights = o_neuron.delta * h_neuron.output
                
                # update weights
                weight_updated = o_weights - 0.5 * errors_total_weights
                print(weight_updated)
        
    def compute_total_deltas_weights(self, index):
        total_delta_weight = 0
        for neuron in nn.output_layer:
            total_delta_weight += neuron.delta * neuron.weights[index]
        return total_delta_weight
    
    def update_hidden_weights(self):
        for i, neuron in enumerate(self.hidden_layer):
            for j in range(0, len(neuron.weights)):
                total_deltas_weights = self.compute_total_deltas_weights(i)            
                error_total_weight = total_deltas_weights * neuron.derivative_activation() * inputs[j]

                # update weights
                weight_updated = neuron.weights[j] - 0.5* error_total_weight
                print(weight_updated)
        
    def feed_forward(self, inputs, neurons):
        outputs = []
        for neuron in neurons:
            outputs.append(neuron.calculate_output(inputs))
        return outputs
    
    def calculate_error(self, target, output):
        return (0.5)*(target - output)**2
    
    def calculate_total_error(self, targets, outputs):
        total_error = 0.0
        for target, output in zip(targets, outputs):
            total_error += self.calculate_error(target, output)
        return total_error
    

Para testar, utilizaremos os seguintes dados do exemplo.

In [121]:
inputs = [.05, .10]
targets = [.01, .99]

nn = NeuralNetwork(num_inputs=2, num_hidden=2, num_outputs=2)
nn_out = nn.forward_pass(inputs)
nn.calculate_total_error(nn_out, targets)

0.2983711087600027

In [122]:
print("----- Hidden Layer outputs -----")
for n in nn.hidden_layer:
    print(n.output)  

----- Hidden Layer outputs -----
0.5932699921071872
0.596884378259767


In [123]:
print("----- Output Layer outputs -----")
for n in nn.output_layer:
    print(n.output)  

----- Output Layer outputs -----
0.7513650695523157
0.7729284653214625


Apos o backward pass os pesos da Output Layer devem ser alterados para os seguintes valores:
[0.35891648, 0.408666186], [ 0.511301270, 0.561370121]

In [124]:
print("----- Output Layer Weights -----")    
for n in nn.output_layer:
    print(n.weights)    

----- Output Layer Weights -----
[0.4, 0.45]
[0.5, 0.55]


Apos o backward pass os pesos da Hidden Layer devem ser alterados para os seguintes valores:
[,], [,]

In [125]:
print("----- Hidden Layer Weights -----")    
for n in nn.hidden_layer:
    print(n.weights)

----- Hidden Layer Weights -----
[0.15, 0.2]
[0.25, 0.3]


In [126]:
nn.compute_output_deltas(targets)
nn.update_output_weights()

0.35891647971788465
0.4086661860762334
0.5113012702387375
0.5613701211079891


In [127]:
nn.update_hidden_weights()

0.1497807161327628
0.19956143226552567
0.24975114363236958
0.29950228726473915
